In [1]:
from kafka import KafkaConsumer
import time
from time import sleep
from json import loads
from json import dumps
import pandas as pd
import json
import pymysql

In [ ]:
def consumer_2(p1,p2,p3,p4):
    try:
        connection = pymysql.connect(host='localhost',
                                     database='mydb',
                                     user='hamz',
                                     password='453565')


        table = """INSERT INTO movingaverage (Price, Buy, Action, TakeProfit) 
           VALUES (%s, %s, %s, %s)"""

        cursor = connection.cursor()
        result = cursor.execute(table,(p1,p2,p4,p3))
        connection.commit()

        print("Record Inserted Successfully.")

    except pymysql.Error as error:
        print("Failed to insert record in MySQL: {}".format(error))
    finally:
        if connection is not None:
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [2]:
consumer = KafkaConsumer('topic2',
                        bootstrap_servers=['localhost:9092'],
                        value_deserializer=lambda x:json.loads(x.decode('utf-8')))

In [3]:
consumer = KafkaConsumer(
    'topic2',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [4]:
# Define moving average window size
window_size = 21

# Define take profit threshold
take_profit = 85.00

# Initialize empty list to store price data
price_data = []

# Initialize flag for take profit
take_profit_flag = False


In [10]:
# Set start time and time limit for consumer
start_time = time.time()
time_limit = 20  # in seconds

In [11]:
# Monitor stock prices
for message in consumer:
    # Check if time limit has been reached
    elapsed_time = time.time() - start_time
    if elapsed_time >= time_limit:
        break
    stock_data = message.value
    price = stock_data['Adj Close:CL=F']
    
    # Add price to price data list
    price_data.append(price)
    
    # Calculate moving average
    if len(price_data) >= window_size:
        moving_average = pd.Series(price_data).rolling(window_size).mean().iloc[-1]
    else:
        moving_average = price
        
    # Check if the current price is above or below the moving average
    if price > moving_average:
        # Trigger a buy order if the price is above the moving average
        print(f'Price above moving average (${price}), triggering buy order...')
        consumer_2(price,'Yes','NO','NO')
        # Place code here to trigger a buy order using your trading API
        
        # Set take profit flag to True
        take_profit_flag = True
        
    elif price < moving_average and take_profit_flag:
        # Trigger a sell order if the price is below the moving average and take profit flag is True
        print(f'Price below moving average (${price}), triggering sell order...')
        # Place code here to trigger a sell order using your trading API
        consumer_2(price,'No','Yes','NO')
        # Reset take profit flag to False
        take_profit_flag = False
        
    else:
        print(f'Price within acceptable range (${price}), no action taken.')
        consumer_2(price,'NO','NO','YES')
    # Wait for 10 seconds before processing the next message
    time.sleep(1)

Price within acceptable range ($80.68000030517578), no action taken.
Price within acceptable range ($80.66000366210938), no action taken.
Price above moving average ($80.83999633789062), triggering buy order...
Price above moving average ($80.79000091552734), triggering buy order...
Price above moving average ($80.93000030517578), triggering buy order...
Price below moving average ($80.18000030517578), triggering sell order...
Price within acceptable range ($80.19000244140625), no action taken.
Price within acceptable range ($80.26000213623047), no action taken.
Price within acceptable range ($80.3499984741211), no action taken.
Price above moving average ($80.70999908447266), triggering buy order...
Price above moving average ($80.5999984741211), triggering buy order...
Price below moving average ($80.41999816894531), triggering sell order...
Price within acceptable range ($80.27999877929688), no action taken.
Price above moving average ($80.56999969482422), triggering buy order...
Pr